In [1]:
import pandas as pd
import random 
cost_matrix = pd.read_csv('Cost_matrix10.csv',header=None)
flow_matrix = pd.read_csv('Flow_matrix10.csv',header=None)

In [2]:
def generate_hubs(number_of_nodes,number_of_hubs):
    hubs = []
    while len(hubs)< number_of_hubs:
        random_number = random.randint(0,9)
        if random_number not in hubs:
            hubs.append(random_number)
    return hubs

In [3]:
def calculate_total_cost(cost_matrix,flow_matrix,alpha,solution):
    total_cost = 0
    for i in range(len(flow_matrix)):
        for j in range(len(flow_matrix)):
            collection_cost = flow_matrix[i][j] * cost_matrix[i][solution[i]]
            transportation_cost = flow_matrix[i][j] * cost_matrix[solution[i]][solution[j]] * alpha
            distribution_cost = flow_matrix[i][j] * cost_matrix[solution[j]][j]
            cost = collection_cost + transportation_cost + distribution_cost
            total_cost += cost
    return total_cost

In [5]:
def get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs):
    solution = [0 for _ in range(len(flow_matrix))]
    #randomly select certain number of hubs
    hubs = generate_hubs(len(flow_matrix),number_of_hubs)
    '''#allocate nodes to their nearest hubs
    for i in range(len(cost_matrix)):
        target_hub = hubs[0]
        for hub in hubs:
            if cost_matrix[i][hub]< cost_matrix[i][target_hub]:
                target_hub = hub
        solution.append(target_hub)'''
    #randomly allocate nodes to hubs
    for i in range(len(flow_matrix)):
        if i not in hubs:
            random_number = random.randint(0,number_of_hubs-1)
            solution[i] = hubs[random_number]
        else:
            solution[i] = i
    # calculate total_cost follow Spoke-Hub-Hub-spoke strategy
    whole_cost = calculate_total_cost(cost_matrix,flow_matrix,alpha,solution)
    return hubs,solution,whole_cost

In [10]:
def Tabu_Search(cost_matrix,flow_matrix,alpha,number_of_hubs):
    current_hubs,current_solution,whole_cost = get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs)
    best_solution = {}
    best_solution['solution'] = current_solution.copy()
    best_solution['cost'] = whole_cost
    tabulist = []
    interation = 0
    while interation < 24:
        neighbours = []
        for i in range(len(flow_matrix)):
            if i not in current_hubs and i not in tabulist:
                for hub in current_hubs:
                    if hub != current_solution[i]:
                        neighbour_solution = current_solution.copy()
                        neighbour_solution[i] = hub
                        neighbour_dict = {}
                        neighbour_dict['solution'] = neighbour_solution.copy()
                        neighbour_dict['node'] = i
                        neighbour_dict['cost'] = calculate_total_cost(cost_matrix,flow_matrix,alpha,neighbour_solution)
                        neighbours.append(neighbour_dict)
        best_neighbour = sorted(neighbours,key = (lambda neighbour: neighbour['cost']))[0].copy()
        #refresh tabu list
        if interation < 5:
            tabulist.append(best_neighbour['node'])
        else:
            tabulist[interation%5] = best_neighbour['node']
        #refresh current solution
        current_solution = best_neighbour['solution'].copy()
        #refresh best solution 
        if best_neighbour['cost'] < best_solution['cost']:
            best_solution['solution'] = best_neighbour['solution'].copy()
            best_solution['cost'] = best_neighbour['cost']
        #refresh move
        interation += 1
        print(interation)
        print('current solution:')
        print(current_solution)
        print('best_solution:')
        print(best_solution)
        print('neighbours:')
        print(neighbours)
        print('tabu list:')
        print(tabulist)
        print('------------------------')

In [11]:
Tabu_Search(cost_matrix,flow_matrix,0.3,3)

1
current solution:
[3, 9, 9, 3, 3, 9, 6, 6, 3, 9]
best_solution:
{'solution': [3, 9, 9, 3, 3, 9, 6, 6, 3, 9], 'cost': 1110082517.9652205}
neighbours:
[{'solution': [6, 9, 9, 3, 3, 9, 6, 6, 9, 9], 'node': 0, 'cost': 1330418484.64006}, {'solution': [9, 9, 9, 3, 3, 9, 6, 6, 9, 9], 'node': 0, 'cost': 1327468367.20846}, {'solution': [3, 6, 9, 3, 3, 9, 6, 6, 9, 9], 'node': 1, 'cost': 1319610601.6696205}, {'solution': [3, 3, 9, 3, 3, 9, 6, 6, 9, 9], 'node': 1, 'cost': 1254348631.78024}, {'solution': [3, 9, 6, 3, 3, 9, 6, 6, 9, 9], 'node': 2, 'cost': 1311340419.4252203}, {'solution': [3, 9, 3, 3, 3, 9, 6, 6, 9, 9], 'node': 2, 'cost': 1166211383.1140401}, {'solution': [3, 9, 9, 3, 6, 9, 6, 6, 9, 9], 'node': 4, 'cost': 1380561718.5201998}, {'solution': [3, 9, 9, 3, 9, 9, 6, 6, 9, 9], 'node': 4, 'cost': 1389040853.0539997}, {'solution': [3, 9, 9, 3, 3, 6, 6, 6, 9, 9], 'node': 5, 'cost': 1306062812.2156208}, {'solution': [3, 9, 9, 3, 3, 3, 6, 6, 9, 9], 'node': 5, 'cost': 1174611721.7390807}, {'so

In [104]:
get_initial_solution(cost_matrix,flow_matrix,0.3,3)

([0, 3, 8], [0, 0, 3, 3, 8, 3, 8, 3, 8, 3], 891156958.3745399)